# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [44]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


In [45]:

# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [46]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    temperature=0
)
print("MOLTBOOK_API_KEY:", bool(MOLTBOOK_API_KEY))
print("VERTEX_API_KEY:", bool(GEMINI_VERTEX_API_KEY))

MOLTBOOK_API_KEY: True
VERTEX_API_KEY: True


# Create a moltbook account for your agent

In [47]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [48]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155244663)

'68519742'

In [36]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "Rui_68519742", "description": "Va"}'

{"statusCode":409,"message":"Agent name already taken","timestamp":"2026-02-25T16:27:49.604Z","path":"/api/v1/agents/register","error":"Conflict"}

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [49]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

def _safe_json(r):
    try:
        return r.json()
    except Exception:
        return {"status_code": r.status_code, "text": r.text}

# ---------- list_submolts ----------
@tool
def list_submolts(limit: int = 50) -> dict:
    """List available submolts. Use this tool to find a submolt by name."""
    r = requests.get(
        f"{BASE_URL}/submolts",
        headers=HEADERS,
        params={"limit": limit},
        timeout=15
    )
    return _safe_json(r)
@tool
# ---------- get_submolt ----------
def get_submolt(submolt: str) -> dict:
    """Get one submolt by name (example: 'ftec5660')."""
    submolt = submolt.replace("/m/", "").strip()
    r = requests.get(
        f"{BASE_URL}/submolts/{submolt}",
        headers=HEADERS,
        timeout=15
    )
    return _safe_json(r)
# ---------- subscribe_submolt ----------
@tool
def subscribe_submolt(submolt: str) -> dict:
    """Subscribe to a submolt by name (example: 'ftec5660')."""
    submolt = submolt.replace("/m/", "").strip()
    r = requests.post(
        f"{BASE_URL}/submolts/{submolt}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    return _safe_json(r)
# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()


In [50]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
- list_submolts
- subscribe_submolt
- get_submolt
"""


# A simple agent to interact with moltbook

In [51]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any


def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
    )

    tools = [
        get_feed,
        search_moltbook,
        list_submolts,
        get_submolt,
        subscribe_submolt,
        create_post,
        comment_post,
        upvote_post,
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



In [55]:
# You need to complte the tool set so that your agent can find the submolt
POST_ID = "47ff50f3-8255-4dee-87f4-2c3637c7351c"
moltbook_agent_loop("find submolt named ftec5660")
print(moltbook_agent_loop("subscribe to submolt ftec5660"))
print(moltbook_agent_loop(
    f"upvote post {POST_ID} and comment a short thoughtful message about learning AI agents in FTEC5660"
))

/tmp/ipython-input-552063350.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[16:43:11] [INIT] Starting Moltbook agent loop
[16:43:12] [HUMAN] find submolt named ftec5660
[16:43:12] [TURN] Turn 1/8 started
[16:43:13] [LLM] Model responded
[16:43:13] [LLM.CONTENT] <empty>
[16:43:13] [LLM.TOOL_CALLS] [
  {
    "name": "get_submolt",
    "args": {
      "submolt": "ftec5660"
    },
    "id": "c4a0512e-6418-4d37-aa1a-9b5de4851323",
    "type": "tool_call"
  }
]
[16:43:13] [TOOL] [1] Calling `get_submolt`
[16:43:13] [TOOL.ARGS] {
  "submolt": "ftec5660"
}
[16:43:13] [TOOL.RESULT] get_submolt finished (success) in 0.18s
[16:43:13] [TOOL.OUTPUT] {
  "success": true,
  "submolt": {
    "id": "fb94de2f-6a69-4105-9118-2c27da9c21df",
    "name": "ftec5660",
    "display_name": "FTEC5660",
    "description": "Discussions, notes, and insights for the FTEC5660 course. AI, agents, experiments, and shared learning.",
    "creator_id": "f8a80401-bdff-4c0d-bc92-076af920cc2f",
    "created_by": {
      "id": "f8a80401-bdff-4c0d-bc92-076af920cc2f",
      "name": "BaoNguyen",
     